In [ ]:
#importar librerias y peticion al servidor
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='http://www.rmiq.org/ojs311/index.php/rmiq/issue/archive'
url_root= 'http://www.rmiq.org/ojs311/index.php/rmiq/issue/archive'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
#primera funcion para obtener links de los volumenes 
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div',class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol

In [ ]:
#Aqui se realiza una lista donde la variable vol contendra los links de los articulos dentro de cada volumen,
#con estos links obtenidos, se implemeta un ciclo para obtener todos los links de los articulos
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('ul', class_='cmp_article_list articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol2+=vol1

In [ ]:
#imprime los articulos
vol2

In [ ]:
#Lee y guarda cada uno de los links obtenidos
def get_url_items(sopa,url):
    box1 = soup1.find('ul', class_='cmp_article_list articles') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [ ]:
#funcion para encontrar la paginacion e ir iterando en cada una y obtener los links a scrapear en ese momento
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

In [ ]:
#esta funcion lo que hace es un listado de los articulos que se encontraron y dar el numero exacto
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
#Se iran tomado uno a uno los links para scrapearlos
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}#diccionario que no va acumulando la imformacion para el tada frame
    r=requests.get(url)
    tire='Revista Mexicana de Ingeniería Química'
    a='VII. Ingenierías'
    tem='Ingeniería Química y sus interfaces con otras disciplinas de la Ingeniería: Termodinámica Catálisis, Cinética y Reactores; Simulación y Control; Fenómenos de Transporte; Seguridad; Ingeniería de Procesos; Biotecnología; Ingeniería de Alimentos'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('p').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
#esta funcion comienza a scrapear cada uno de los articulos para obtener la informacion requerida
list_scraper=list_scraper[0:469]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
#una vez terminado el scraping se imprimen los datos en un dataframe
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
#Si se detecta alguna infotmacion repetida o que no es importante con esta funcion se elimina la fila
df=df_catalogo.drop(df_catalogo.index[[47, 303]])
df

In [ ]:
#ya teniendo la tabla con la informacion deseada se pasa a guardar como un archivo CSV
df.to_csv('Revista005.csv', index=False)